<a href="https://colab.research.google.com/github/ThousandAI/Application-of-AI/blob/main/class08/LSTM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import torch
import torchvision
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler
from tqdm import tqdm
from sklearn.metrics import mean_absolute_percentage_error ,mean_squared_error

In [2]:
class StockDataset(Dataset):
    def __init__(self,start,end,seq_length,dim,mode,train_dataset=None):
        self.seq_length = seq_length
        self.data = np.array(pd.read_csv("/content/drive/MyDrive/Colab-Notebooks/Application-of-AI/class08/stock.csv"))[start:end,:-1]
        if mode == 'train':
            self.sc = StandardScaler()
            self.sc_close = StandardScaler()
            self.sc_close.fit(self.data[:,1:2])
            self.data = self.sc.fit_transform(self.data)
        elif mode == 'test':
            self.data = train_dataset.sc.fit_transform(self.data)
        self.n_samples = len(self.data) - self.seq_length
        self.x = np.zeros((self.n_samples,seq_length,dim))
        self.y = np.zeros((self.n_samples, 1))
        for i in range(len(self.data)-self.seq_length):
            self.x[i,:,:] = self.data[i:i+self.seq_length,:]
            self.y[i,0] = self.data[i+seq_length,1]

    def __getitem__(self, index):
        return self.x[index], self.y[index]

    def __len__(self):
        return self.n_samples


In [3]:
# hyper parameters
input_size = 11 # 股價維度
seq_length = 90 # 序列長度
hidden_size = 256
num_layers = 2
epochs = 100
batch_size = 16
learning_rate = 1e-3
start = 0
end = 5000
test_start = 5000
test_end = 5900
# device
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [4]:
# dataset
train_dataset = StockDataset(start=start,end=end,seq_length=seq_length,dim=input_size,mode="train")
test_dataset = StockDataset(start=test_start,end=test_end,seq_length=seq_length,dim=input_size,mode="test",
                            train_dataset=train_dataset)
train_dataloader = DataLoader(dataset=train_dataset, batch_size=batch_size, shuffle= True)
test_dataloader = DataLoader(dataset=test_dataset, batch_size=batch_size, shuffle=False)

In [5]:
class LSTM(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers):
        super(LSTM, self).__init__()
        self.num_layers = num_layers
        self.hidden_size = hidden_size
        self.lstm = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_size,1)

    def forward(self, x):
        # out: tensor of shape (batch_size, seq_length, hidden_size)
        # out: (n, 15, 768)
        out,_ = self.lstm(x)


        # Decode the hidden state of the last time step
        out = out[:, -1, :]
        # out: (n, 768)

        out = self.fc(out)
        # out: (n, 1)
        return out

In [6]:
lstm = LSTM(input_size=input_size, hidden_size=hidden_size, num_layers=num_layers).to(device)
optimizer = torch.optim.Adam(lstm.parameters(), lr=learning_rate)
criterion = nn.MSELoss()

In [7]:
# Train the model

mape_loss = 0
rmse_loss = 0
for epoch in tqdm(range(epochs)):
    for i, (inputs, labels) in enumerate(train_dataloader):
        inputs = inputs.type(torch.FloatTensor).to(device)
        labels = labels.type(torch.FloatTensor).to(device)

        # Forward pass
        outputs = lstm(inputs)
        loss = criterion(outputs, labels)

        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    print(f'Epoch [{epoch + 1}/{epochs}], Loss: {loss.item():.8f}')

  1%|          | 1/100 [00:03<05:14,  3.17s/it]

Epoch [1/100], Loss: 0.00411349


  2%|▏         | 2/100 [00:06<05:17,  3.23s/it]

Epoch [2/100], Loss: 0.00172368


  3%|▎         | 3/100 [00:09<05:16,  3.26s/it]

...

Epoch [98/100], Loss: 0.00149321


 99%|█████████▉| 99/100 [04:05<00:02,  2.44s/it]

Epoch [99/100], Loss: 0.00047735


100%|██████████| 100/100 [04:07<00:00,  2.48s/it]

Epoch [100/100], Loss: 0.00079641


In [8]:
with torch.no_grad():
    count  = 0
    for ind,(inputs,labels) in enumerate(test_dataloader):
        inputs = inputs.type(torch.FloatTensor).to(device)
        labels = labels.type(torch.FloatTensor).to(device)

        outputs = lstm(inputs)
        y_pred = train_dataset.sc_close.inverse_transform(outputs.cpu())
        y_true = train_dataset.sc_close.inverse_transform(labels.cpu())
        mape_loss += mean_absolute_percentage_error(y_true, y_pred)
        rmse_loss += mean_squared_error(y_true, y_pred,squared=False)
        count += 1
    print(f"mape: {mape_loss / count}")
    print(f"rmse: {rmse_loss / count}")
    print(f"count: {count}")

mape: 0.0474502483153465
rmse: 18.155281804723497
count: 51
